In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [ ]:
with open('../config.json') as file:
    app_data = json.load(file)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials( \
                     client_id=app_data['client_id'], \
                     client_secret=app_data['client_secret']))

In [ ]:
all_years = pd.read_csv('../data/Billboard_Audio_Features.csv')
all_years.head(1)

In [ ]:
all_years = pd.read_csv('../data/Billboard_Audio_Features.csv')
all_years['Artist IDs'] = all_years['Artist IDs'].str.lstrip('[').str.rstrip(']')
all_years['Primary Artist ID'] = all_years['Artist IDs'].str.split(',', expand = True)[0].str.strip("'")
all_years['Genres'] = pd.Series(dtype=object)
all_with_genres = pd.DataFrame(columns=all_years.columns)

In [ ]:
all_years = all_years.tail(3408)
for i, row in all_years.iterrows():
    artist = sp.artist(row['Primary Artist ID'])
    row['Genres'] = ", ".join(artist["genres"])
    all_with_genres = all_with_genres.append(row, ignore_index= True)

In [ ]:
all_with_genres.to_csv('../data/Billboard_Audio_Features.csv', index=False)

In [11]:
data = pd.read_csv('../data/Billboard_Audio_Features.csv')
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5746 entries, 0 to 6068
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Album ID           5746 non-null   object 
 1   Artist IDs         5746 non-null   object 
 2   Artist(s)          5746 non-null   object 
 3   ID                 5746 non-null   object 
 4   Popularity         5746 non-null   float64
 5   Query              5746 non-null   object 
 6   Rank               5746 non-null   float64
 7   Release Date       5746 non-null   object 
 8   Song               5746 non-null   object 
 9   Year               5746 non-null   float64
 10  danceability       5746 non-null   float64
 11  energy             5746 non-null   float64
 12  key                5746 non-null   int64  
 13  loudness           5746 non-null   float64
 14  mode               5746 non-null   int64  
 15  speechiness        5746 non-null   float64
 16  acousticness       5746 

In [40]:
# rewrite with np.where or apply
new_data = pd.DataFrame(columns=data.columns)

for i, row in data.iterrows():
    genre = np.nan
    genres = row["Genres"]
    if "country" in genres:
        genre = "country"
    elif "disco" in genres:
        genre = "disco"
    elif "edm" in genres or "electro" in genres:
        genre = "edm"
    elif "r&b" in genres or "soul" in genres:
        genre = "soul/r&b"
    elif "hip hop" in genres or "rap" in genres:
        genre = "hip hop/rap"
    elif "alternative" in genres or "indie" in genres:
        genre = "alternative/indie"
    elif "rock" in genres or "pop" in genres:
        rock = 0
        pop = 0
        for g in genres.split(','):
            if "rock" in g:
                rock += 1
            if "pop" in g:
                pop += 1
        if rock >= pop:
            genre = "rock"
        else:
            genre = "pop"
    row["Top Genre"] = genre
    new_data = new_data.append(row)

new_data['Top Genre'].value_counts()

In [42]:
new_data.to_csv('../data/Billboard_Audio_Features.csv', index=False)

In [39]:
# create dictionary of word frequency in genre
categories = {}
test = new_data[new_data["Top Genre"] == 'disco']
for genre in test['Genres']:
    #genre = genre.replace(",", "")
    genres = genre.split(',')
    words = [x.strip() for x in genres]
    uniques = []
    for word in words:
        if word not in uniques:
            uniques.append(word)
    for word in uniques:
        if word in categories:
            categories[word] += 1
        else:
            categories[word] = 1

# sort dictionary by frequency to get most common overall genre categories
list({k: v for k, v in sorted(categories.items(), key=lambda item: item[1], reverse=True)}.items())[:15]

[('disco', 404),
 ('quiet storm', 272),
 ('motown', 255),
 ('funk', 221),
 ('soul', 207),
 ('soft rock', 173),
 ('mellow gold', 143),
 ('adult standards', 133),
 ('classic soul', 118),
 ('post-disco', 115),
 ('new wave pop', 79),
 ('urban contemporary', 57),
 ('bubblegum pop', 50),
 ('brill building pop', 49),
 ('southern soul', 45)]

In [ ]:
import operator

# create dictionary of word frequency in genre
categories = {}
for genre in data['Genres']:
    genre = genre.replace(",", "")
    words = genre.split(' ')
    count = {}
    for word in words:
        if word in count:
            count[word] += 1
        else:
            count[word] = 1
    word = max(count.items(), key=operator.itemgetter(1))[0]
    if word in categories:
        categories[word] += 1
    else:
        categories[word] = 1

# sort dictionary by frequency to get most common overall genre categories
list({k: v for k, v in sorted(categories.items(), key=lambda item: item[1], reverse=True)}.items())